In [1]:
import torch
from torch import nn

/data/wangyidan-slurm/anaconda3/envs/pt4dm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 对nn.Module的再认识

### parameter & submodel
* net._parameters
* net._modules
* net.named_parameters()
* net.named_modules()

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.param1 = nn.Parameter(torch.randn(3,3)) # 当前模块的要学习的参数
        self.submodel1 = nn.Linear(3,4) #  当前模块的子模块
        
    def forward(self,x):
        x = self.param1.mm(x) # mm 矩阵乘法
        x = self.submodel1(x)
        
        return x
net = Net()
net

Net(
  (submodel1): Linear(in_features=3, out_features=4, bias=True)
)

In [3]:
# 查看模型的子模型
net._modules

OrderedDict([('submodel1', Linear(in_features=3, out_features=4, bias=True))])

In [4]:
# 查看模型的训练参数
net._parameters # 只有param1，3*3

OrderedDict([('param1',
              Parameter containing:
              tensor([[-0.1432, -0.3114,  0.0510],
                      [ 0.4637,  1.1742,  1.2909],
                      [ 0.7148,  0.8683,  2.0601]], requires_grad=True))])

In [5]:
net.param1 == net._parameters['param1']

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [6]:
# named_parameters()则会返回所有的训练参数
for name,param in net.named_parameters():
    print(name,param.size())

param1 torch.Size([3, 3])
submodel1.weight torch.Size([4, 3])
submodel1.bias torch.Size([4])


In [7]:
# named_modules()
for name,submodel in net.named_modules():
    print(name,submodel)

 Net(
  (submodel1): Linear(in_features=3, out_features=4, bias=True)
)
submodel1 Linear(in_features=3, out_features=4, bias=True)


### _buffers
`_buffers`：缓存。如batchnorm使用momentum机制，每次前向传播需用到上一次前向传播的结果。

In [8]:
# _buffers
bn = nn.BatchNorm1d(num_features=2)
x = torch.randn(3,2)
output = bn(x)
bn._buffers

OrderedDict([('running_mean', tensor([-0.0312, -0.0080])),
             ('running_var', tensor([0.9707, 1.1294])),
             ('num_batches_tracked', tensor(1))])

### module 与 子module
- named_childen
- named_modules


### train 与 eval
`model.training`
对于batchnorm、dropout、instancenorm等在训练和测试阶段行为差距巨大的层，如果在测试时不将其training值设为True，则可能会有很大影响。
调用`model.train()`函数，它会将当前module及其子module中的所有training属性都设为True，相应的，`model.eval()`函数会把training属性都设为False。

In [9]:
# 此时是训练阶段
x = torch.arange(0,12).view(3,4).float()
model = nn.Dropout() # p默认=0.5，一半的数据会被设置为0
model(x)

tensor([[ 0.,  0.,  0.,  6.],
        [ 8., 10., 12.,  0.],
        [16., 18., 20.,  0.]])

In [10]:
model.training = False # 非训练阶段，dropout什么都不做
model(x)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [11]:
print(net.training, net.submodel1.training)
net.eval()
print(net.training, net.submodel1.training)
net.train()
print(net.training, net.submodel1.training)

True True
False False
True True



### 钩子(?)
钩子函数主要用在获取某些中间结果的情景，如中间某一层的输出或某一层的梯度。
- `register_forward_hook`
- `register_backward_hook`

下面考虑一种场景，有一个预训练好的模型，需要提取模型的某一层（不是最后一层）的输出作为特征进行分类，但又不希望修改其原有的模型定义文件，这时就可以利用钩子函数。下面给出实现的伪代码。
```python
model = VGG()
features = torch.Tensor()
def hook(model,input,output):
    features.copy_(output.data)

handle = model.layer8.register_forward_hook(hook) # !!
_ = model(input)
# 用完后删除
handle.remove()
```



### nn.Module的构造函数(?)
- `__getattr__`
-`__setattr__`

nn.Module实现了自定义的`__setattr__`函数，当执行`module.name=value`时，会在`__setattr__`中判断value是否为`Parameter`或`nn.Module`对象，如果是则将这些对象加到`_parameters`和`_modules`两个字典中，而如果是其它类型的对象，如`Variable`、`list`、`dict`等，则调用默认的操作，将这个值保存在`__dict__`中。

nn.Module能够自动组织参数parameter和子模块module的原因

In [14]:
module = nn.Module()
module.param = nn.Parameter(torch.ones(2,2)) # setattr，添加参数
module._parameters

OrderedDict([('param',
              Parameter containing:
              tensor([[1., 1.],
                      [1., 1.]], requires_grad=True))])

In [17]:
submodule1 = nn.Linear(2,2)
submodule2 = nn.Linear(2,2)
module_list = [submodule1,submodule2]

In [18]:
module.submodules = module_list # setattr，添加子模型，但因为包含在list对象里，不会被识别到
module.submodule = submodule1 # 能够被识别到

print('_modules: ',module._modules)
print('__dict__["submodules"]:',module.__dict__.get('submodules'))

_modules:  OrderedDict([('submodule', Linear(in_features=2, out_features=2, bias=True))])
__dict__["submodules"]: [Linear(in_features=2, out_features=2, bias=True), Linear(in_features=2, out_features=2, bias=True)]


In [19]:
module_list = nn.ModuleList(module_list)
module.submodules = module_list
print('ModuleList is instance of nn.Module: ',isinstance(module_list,nn.Module))
print('_modules: ',module._modules)
print('__dict__["submodules"]:',module.__dict__.get('submodules'))

ModuleList is instance of nn.Module:  True
_modules:  OrderedDict([('submodule', Linear(in_features=2, out_features=2, bias=True)), ('submodules', ModuleList(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
))])
__dict__["submodules"]: None


In [20]:
module

Module(
  (submodule): Linear(in_features=2, out_features=2, bias=True)
  (submodules): ModuleList(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): Linear(in_features=2, out_features=2, bias=True)
  )
)

In [21]:
getattr(module,'training')

True

In [22]:
getattr(module,'param')

Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


### 保存模型
- `state_dict()` && `load_state_dict()`
- `save()` && `load()`

In [12]:
import os
path = 'save_model/'
torch.save(net.state_dict(),os.path.join(path,'net.pth'))

In [13]:
# 加载模型
net2 = Net()
net2.load_state_dict(torch.load(os.path.join(path,'net.pth')))

<All keys matched successfully>

In [ ]:
### GPU上运算
- model = model.cuda()：将模型的所有参数转存到GPU
- input.cuda()：将输入数据也放置到GPU上
